In [1]:
from scipy.stats import dirichlet
import numpy as np

In [2]:

from scipy.stats import multinomial

def make_dataset(n, alpha, beta):
    xs = []
    for k, alpha_k in enumerate(alpha):
        n_k = int(n * alpha_k)
        x = multinomial.rvs(n=16, p=beta[k], size=n_k)
        xs.append(x)
    return np.vstack(xs)

In [3]:
alpha = np.array([1/3,1/3,1/3])
beta = np.array([[0.1, 0.1, 0.8], [0.1, 0.8, 0.1], [0.8, 0.1, 0.1]])

In [4]:
X = make_dataset(10, alpha,beta )

In [5]:
seed = 42
rng = np.random.default_rng(seed = 42)

In [6]:
def init_params(X, K):
    C = X.shape[1]
    weights = np.random.randint(1, 20, K)
    alpha = weights / weights.sum()
    beta = dirichlet.rvs([2 * C] * C, K)
    return alpha, beta

In [7]:
x = rng.uniform(-0,1, size = 3)

In [8]:
(x/x.sum()).sum()

0.9999999999999998

In [9]:
a, b= init_params(X, 3)

In [10]:
import math

In [11]:
def multinomial_probability(x_i, beta_k):
    n_i = x_i.sum()
    d = 1
    for i in range(len(x_i)):
        d*= (beta_k[i]**x_i[i]/math.factorial(x_i[i]))

    return n_i*d
        



In [12]:
def multinomial_pmf(counts, beta, log = False):
        n = counts.sum(axis= 1)
        m = multinomial(n, beta)
        if log:
            return m.logpmf(counts)
        return m.pmf(counts)

In [13]:
X.sum(axis = 0)

array([49, 44, 51])

In [14]:
multinomial_pmf(X, beta[0])

array([1.05553116e-01, 3.07863256e-02, 1.87604171e-02, 5.60000000e-14,
       3.72736000e-10, 7.87218432e-07, 5.82400000e-12, 1.07520000e-11,
       1.07520000e-11])

In [15]:
import scipy

In [16]:

def e_step(data, n_clusters, beta, pi):
    likelihood = np.zeros((data.shape[0], n_clusters))
    #responsibility = np.zeros_like(likelihood)
    for i in range(n_clusters):
        
        betai = beta[i] 
        # if any(np.isnan(mui)):
        #     new_mu, new_sigma, new_pi = init(data, n_clusters)
        #     mui = new_mu[i]
        #     covi = new_sigma[i]
        

        #normal_dist = scipy.stats.multivariate_normal(mean = mu[i], cov = sigma[i])
        likelihood[:,i] = multinomial_pmf(counts = data,  beta=betai)
    numerator = (likelihood*pi) + 1e-18
    denominator = numerator.sum(axis = 1)[:, np.newaxis] 
    denominator = denominator 
    

    responsibility = numerator/denominator
    return responsibility

In [17]:
r = e_step(X, 3, b, a)

In [18]:
def m_step(data,beta, pi, n_clusters, responsibility):
    

    for i in range(n_clusters):
        weight = responsibility[:, [i]]
        total_weight = weight.sum()
        beta[i] = (data * weight).sum(axis=0) / (data * weight).sum()       

        pi[i] = responsibility[:,i].sum()/len(data)
    return np.array(beta), np.array(pi)



In [19]:
m_step(X, b, a, 3, r)

(array([[0.13878429, 0.11066766, 0.75054805],
        [0.65788322, 0.24824154, 0.09387524],
        [0.11830272, 0.68009047, 0.2016068 ]]),
 array([0.35600015, 0.39787134, 0.24612851]))

In [20]:
def log_loss( X, alpha, beta, gamma):
    """
    X: (N x C), matrix of counts
    alpha: (K)  mixture component weights
    beta: (K x C), multinomial categories weights
    gamma: (N x K), posterior probabilities for cluster assignments
    :return:  The variational lower bound value
    """
    loss = 0
    for k in range(beta.shape[0]):
        weights = gamma[:, k]
        loss += np.sum(weights * (np.log(alpha[k]) + multinomial_pmf(X, beta[k], log=True)))
        loss -= np.sum(weights * np.log(weights))
    return -1*loss

In [21]:
log_loss(X, a, b, r)

49.86497763428007

In [22]:
def mmm(data, max_iter, n_clusters, rtol = 1e-4):
    import matplotlib.pyplot as plt
    iter = 0
    pi, beta= init_params(data, n_clusters)
    l = []
    loss = 0
    while iter < max_iter:
        prev_loss = loss
        responsibility = e_step(data, n_clusters, beta, pi)
        beta, pi = m_step(data,beta, pi, n_clusters, responsibility)
        loss = log_loss( X, pi, beta, responsibility)
        l.append(loss)
       
        
        iter += 1
        #print(f'loss: {loss}')
        print('Loss: %f' % loss)
        if iter > 0 and np.abs((loss - prev_loss)) < rtol:
            print(iter)
            break
    return np.argmax(np.array(responsibility), axis = 1), loss
        
        



In [23]:
mmm(X,100, 3)

Loss: 39.539863
Loss: 38.257076
Loss: 38.256472
Loss: 38.256472
4


(array([0, 0, 0, 1, 1, 1, 2, 2, 2], dtype=int64), 38.25647175580019)

In [24]:
from mmm import MultinomialMixtureModel

In [25]:
model = MultinomialMixtureModel(3)
model.fit(X)


iteration 0
Loss: 48.953038
Loss: 38.260000
Loss: 38.256472
3
better loss on iteration 0: 38.2564717606
iteration 1
Loss: 66.946769
Loss: 60.163403
Loss: 58.899773
Loss: 58.575335
Loss: 58.568496
5
iteration 2
Loss: 65.582459
Loss: 61.230000
Loss: 54.732354
Loss: 45.661003
Loss: 38.660855
Loss: 38.256472
Loss: 38.256472
7
better loss on iteration 2: 38.2564717558
iteration 3
Loss: 56.262834
Loss: 42.253433
Loss: 38.414355
Loss: 38.256478
Loss: 38.256472
5
iteration 4
Loss: 51.734269
Loss: 38.376662
Loss: 38.256475
Loss: 38.256472
4
iteration 5
Loss: 54.432526
Loss: 38.922005
Loss: 38.256609
Loss: 38.256472
4
iteration 6
Loss: 58.800221
Loss: 42.624872
Loss: 38.530100
Loss: 38.256490
Loss: 38.256472
5
iteration 7
Loss: 46.493023
Loss: 38.260804
Loss: 38.256472
3
iteration 8
Loss: 68.629497
Loss: 52.506038
Loss: 38.539024
Loss: 38.256491
Loss: 38.256472
5
iteration 9
Loss: 71.633325
Loss: 46.457713
Loss: 38.279797
Loss: 38.256472
Loss: 38.256472
5
better loss on iteration 9: 38.256471755

(38.2564717558002,
 array([0.33333341, 0.33332744, 0.33333915]),
 array([[0.79166652, 0.10416667, 0.10416681],
        [0.12500003, 0.70833575, 0.16666423],
        [0.104167  , 0.10417493, 0.79165807]]),
 array([2, 2, 2, 1, 1, 1, 0, 0, 0], dtype=int64))

In [26]:
X[1].shape
np.array(list(X[1].reshape((1,3))))

array([[ 0,  3, 13]])

In [27]:
X[1]

array([ 0,  3, 13])

In [28]:
X[1].shape == (len(X[1]), )

True

In [29]:
model.score_sample(X)

2.428988280294053e-17